In [79]:
import pandas as pd
import numpy as np
import json
import operator
import itertools 
import matplotlib.pyplot as plt
import re
import subprocess

In [75]:
dataDir="./datasets/"
datasets=[
    "adult",
    "airport",
    "atom",
    "CLASSIFICATION",
    "flights",
    "food",
    "Hospital",
    "inspection",
    "ncvoter",
    "SPStock",
    "tax500k",
]

In [76]:
JARDir="./algorithmJARs/"
algorithms=[
    "Hydra",
    "DCFinder",
    "ADCMiner",
    "FastADC",
]

# Code

## Dataset

In [151]:
class Dataset:
    def __init__(self,file,**args):
        self.columns=pd.read_csv(file,nrows=0).columns
        self.header=[re.match(r'([^\(\)]*)(?:\(| )([^\(\)]*)\)?',col) for col in self.columns]
        self.names=[match[1] for match in self.header]
        typeMap={'String':str,'Integer':float,'Double':float,'int':float,'str':str,'float':float}
        self.types=[typeMap[match[2]] for match in self.header]
        
        self.df=pd.read_csv(file,dtype={col:type for col,type in zip(self.columns,self.types)},**args)
        
    def randRows(self,n):
        ids=np.random.randint(0,len(self.df),n)
        return self.df.iloc[ids]
    def randFields(self,n):
        return pd.DataFrame({col:dfs[col].iloc[list(np.random.randint(0,len(dfs),n))].values for dfs in [self.df] for col in dfs.columns})

    def buildPLIs(self):
        self.PLI= {col:self.df.groupby(by=col).groups for col in self.df}
    def shuffle(self):
        self.df=self.randFields(len(self.df))

## DenialConstraintResult

In [168]:
class Operator:
    def __init__(self,func,expFunc) -> None:
        self.func=func
        self.expFunc=expFunc
    def __call__(self,a,b):
        return self.func(a,b)
    def negate(self):
        return Operator(operator.invert(self.func))
    def expected(self,c1,c2):
        return self.expFunc(c1,c2)
    def __repr__(self) -> str:
        return self.func.__name__
    def __eq__(self, other: object) -> bool:
        if isinstance(other, Operator):
            return self.func==other.func
        return False
    def __hash__(self):
        fields=(self.func)
        hash_value = hash(fields)
        return hash_value

def eqExp(l,r):
    vals,freqs=np.unique(l.values,return_counts=True)
    probs=freqs/len(l)
    return np.sum(probs**2)-1/len(l)
   

eq=Operator(operator.eq,eqExp)

def neExp(l,r):
    vals,freqs=np.unique(l.values,return_counts=True)
    probs=freqs/len(l)
    return 1-np.sum(probs**2)
ne=Operator(operator.ne,neExp)

def geExp(l,r):
    vals,freqs=np.unique(l.values,return_counts=True)
    probs=freqs/len(l)
    cumProbs=np.cumsum(probs)
    return np.sum(probs*(1-cumProbs+probs))-1/len(l)
ge=Operator(operator.ge,geExp)

def leExp(l,r):
    vals,freqs=np.unique(l.values,return_counts=True)
    probs=freqs/len(l)
    cumProbs=np.cumsum(probs)
    return np.sum(probs*(cumProbs))-1/len(l)
le=Operator(operator.le,leExp)

def gtExp(l,r):
    vals,freqs=np.unique(l.values,return_counts=True)
    probs=freqs/len(l)
    cumProbs=np.cumsum(probs)
    return np.sum(probs*(1-cumProbs))
gt=Operator(operator.gt,gtExp)

def ltExp(l,r):
    vals,freqs=np.unique(l.values,return_counts=True)
    probs=freqs/len(l)
    cumProbs=np.cumsum(probs)
    return np.sum(probs*(cumProbs-probs))
lt=Operator(operator.lt,ltExp)
operatorMap={
    "EQUAL":eq,
    "UNEQUAL":ne,
    "LESS_EQUAL":le,
    "GREATER_EQUAL":ge,
    "LESS":lt,
    "GREATER":gt
}



class Predicate:
    def __init__(self,l:str,op:Operator,r:str) -> None:
        self.l=l
        self.r=r
        self.op=op
        self.exp=None
    def eval(self,df,t0,t1):
        return self.op(t0[self.l],t1[self.r])
    def expected(self,df):
        if self.exp is None:
            self.exp=self.op.expected(df.df[self.l],None)
        return self.exp
            
    
    def __repr__(self) -> str:
        return '"'+self.l +'" '+self.op.__repr__()+' "'+self.r+'"'
    def __hash__(self):
        fields=(self.l,self.r)
        hash_value = hash(fields)
        return hash_value

    def __eq__(self, other):
        if isinstance(other, Predicate):
            sFields=(self.l,self.op,self.r)
            oFields=(other.l,other.op,other.r)
            return sFields==oFields
        return False


class DenialConstraint:
    def __init__(self,preds) -> None:
        self.preds=preds
    def eval(self,df,t0,t1):
        return sum([pred.eval(df,t0,t1) for pred in self.preds])
    def coverage(self,df,t0s,t1s):
        pos,neg=0,0
        num=self.eval(df,t0s,t1s)
        dclen=len(self.preds)
        pos=(num==dclen).sum()
        neg=(num<dclen).sum()

                
        return neg/(pos+neg)
    def sampleCoverage(self,df,n=None):
        nn=len(df.df)
        if n is None:
            n=nn**2
        t0s = np.random.randint(0,len(df.df),n)
        t1s = np.random.randint(0,len(df.df),n)
        t1s=(t1s+(t1s==t0s)*np.random.randint(1,len(df.df),n))%len(df.df)
        return self.coverage(df,t0s,t1s)
    def expCoverage(self,df):
        return 1-np.prod([pred.expected(df) for pred in self.preds])
    def __repr__(self) -> str:
        return "!["+" & ".join([pred.__repr__() for pred in self.preds])+"]"


class DenialConstraintSet:
    def __init__(self,path,dataset,algorithm) -> None:        
        self.predicates={}
        opmap={"==":eq,"<>":ne,">=":ge,"<=":le,">":gt,"<":lt}
        def getPred(c1,op,c2):
            if (c1,c2,op) not in self.predicates:
                self.predicates[(c1,c2,op)]=Predicate(c1,opmap[op],c2)
            return self.predicates[(c1,c2,op)]
        
        self.DCs=[]
        
        with open(path) as f:
            for line in f:
                line=line.strip()[2:-1] #strip !(...)
                preds=line.split('^')
                regex=r't0\.'+dataset+'\.csv\.([^=><]*)(==|<>|>=|<=|>|<)t1\.'+dataset+'\.csv\.([^=><]*)'
                if algorithm in ['ADCMiner','FastADC']:
                    regex=r't0\.([^=><]*) (==|<>|>=|<=|>|<) t1\.([^=><]*)'
                preds = [getPred(*re.match(regex,pred).groups()) for pred in preds]
                self.DCs.append(DenialConstraint(preds))


# Result generation

## Discover DCs

In [163]:
for dataset in datasets[6:7]:
    for algorithm in algorithms[3:4]:
        command='java -Xmx8g -cp {} Main {} 0.01 10000'.format(JARDir+algorithm+'.jar',dataDir+dataset+".csv")
        result = subprocess.run(command, shell=True)

INPUT FILE: ./datasets/Hospital.csv
ERROR THRESHOLD: 0.01
 [Input] # of Tuples: 10000
 [Input] # of Attributes: 15
 [Predicate] Predicate space size: 30
[FastADC] Pre-process time: 603ms
  [CLUE] # of bits in clue: 15
  [CLUE] task count: 435
 [Evidence] # of evidences: 286
 [Evidence] Accumulated evidence count: 99990000
[FastADC] Evidence time: 799ms
 [AEI Start] 2024-02-04 21:17:09
 [AEI] Violate at most 999900 tuple pairs
  [AEI] Inverting evidences...
  [AEI] Min cover size: 92
  [AEI] Total DC size: 92
  [AEI] Min DC size : 92
[FastADC] AEI time: 71ms
[FastADC] Total computing time: 1473 ms


Lines written to file successfully!


In [169]:
dcs=DenialConstraintSet("output.txt","Hospital","FastADC")

In [171]:
len(dcs.DCs)

92

In [170]:
ds=Dataset("datasets/Hospital.csv")
ds.buildPLIs()

In [172]:
n=10000
t0s = ds.randFields(n)
t1s = ds.randFields(n)

res=[ dc.coverage(ds,t0s,t1s) for dc in dcs.DCs]


In [174]:
[i for i,r in enumerate(res) if r<0.9]

[]

In [157]:
dcs.DCs

[!["ZIP Code(String)" eq "ZIP Code(String)"],
 !["Condition(String)" ne "Condition(String)" & "Measure Code(String)" eq "Measure Code(String)"],
 !["Condition(String)" ne "Condition(String)" & "Measure Name(String)" eq "Measure Name(String)"],
 !["County Name(String)" eq "County Name(String)" & "Sample(String)" eq "Sample(String)"],
 !["County Name(String)" eq "County Name(String)" & "State(String)" ne "State(String)"],
 !["StateAvg(String)" eq "StateAvg(String)"],
 !["Measure Name(String)" eq "Measure Name(String)" & "Sample(String)" eq "Sample(String)"],
 !["County Name(String)" eq "County Name(String)" & "Measure Name(String)" eq "Measure Name(String)"],
 !["Condition(String)" eq "Condition(String)" & "County Name(String)" eq "County Name(String)"],
 !["Condition(String)" eq "Condition(String)" & "Hospital Type(String)" ne "Hospital Type(String)" & "State(String)" eq "State(String)"],
 !["Hospital Type(String)" ne "Hospital Type(String)" & "Measure Code(String)" eq "Measure Code(Str

In [137]:
dcs.DCs[8804].expCoverage(ds)

0.9963751279140795

In [124]:
res[8804]

1.0

In [142]:
ltExp(ds.df['capital-loss float'],ds.df['capital-loss float'])

0.04550623851887257

In [8]:
line='¬(t0.adult.csv.Native-country str<>t1.adult.csv.Native-country str^t0.adult.csv.capital-gain float<>t1.adult.csv.capital-gain float^t0.adult.csv.class str==t1.adult.csv.class str^t0.adult.csv.race str==t1.adult.csv.race str)'
line=line[2:-1] #strip !(...)
preds=line.split('^')


In [89]:
eq(ds.df['age int'],ds.df['age int'])

0        True
1        True
2        True
3        True
4        True
         ... 
32556    True
32557    True
32558    True
32559    True
32560    True
Name: age int, Length: 32561, dtype: bool